In [3]:
%pip -q install google-genai  # Instalar biblioteca Gemini

In [4]:
!pip install -q google-adk    # Instalar framework ADK de agentes da Google

In [2]:
# Importar bibliotecas

import os
from google.colab import userdata
from google import genai                      # Importa SDK
from IPython.display import HTML, Markdown    # Importa funções web HTML

# Define variável de ambiente com API KEY
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
client = genai.Client()

# Define o modelo a ser utilizado
MODEL_ID = "gemini-2.0-flash"

In [5]:
# Declarações iniciais

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [45]:
###############################################
# --- Agente 1: Buscador de Propriedades --- #
###############################################
def agente_busca_propriedade(tipo, orcamento, bairro, cidade, params_adic):
  buscador = Agent(
      name = "agente_busca_propriedade",
      model=MODEL_ID,
      description="Utilizar o Google Search para encontrar ativamente listagens online de imóveis que correspondam aos critérios específicos fornecidos pelo usuário.",
      tools=[google_search],
      instruction="""
        Você será um especialista em procurar propriedades na internet.
        Você realizará a busca de imóveis construindo consultas otimizadas e utilizando a ferramenta (google_search) utilizando os parâmetros informados para obter a lista de imóveis mais relevantes.
        Incluir o tipo de imóvel (tipo), o bairro do imóvel (bairro) e cidade do imóvel (cidade). Usar aspas para frases exatas (ex: "apartamento venda").
        Incorporar a faixa de orçamento sendo o limite do preço até (orcamento).
        Considerar imóveis só para venda e dentro do valor total de compra e da localidade informada nos parâmetros.
        Procurar em sites confiáveis.
        A saída deverá ter o seguinte formato:
        - Coluna 1: ID do imóvel (utilizar um sequencial)
        - Coluna 2: Título do imóvel
        - Coluna 3: Descrição
        - Coluna 4: Preço
        - Coluna 5: Endereço
        - Coluna 6: Cidade
        - Coluna 7: Bairro
        - Coluna 8: Nome da construtora
        - Coluna 9: Telefone de Contato do Vendedor
      """
  )

  dados_propriedade = f"Tipo de Propriedade: {tipo}/n Orçamento em Reais: {orcamento}/n Bairro: {bairro}/n Cidade: {cidade}/n Parâmetros Adicionais: {params_adic}"
  propriedades = call_agent(buscador, dados_propriedade)
  return propriedades


In [54]:
################################################
# --- Agente 2: Analisador de Localizações --- #
################################################
def agente_localizacao(propriedades):

  localizador = Agent(
      name = "agente_localizacao",
      model=MODEL_ID,
      description="Coletar e analisar informações contextuais sobre a área geográfica de um imóvel específico, fornecendo insights sobre a vizinhança, acessibilidade e serviços próximos utilizando a ferramenta (google_search)",
      tools=[google_search],
      instruction="""
        Você atuará como um especialista em análise de informações contextuais de uma área geográfica focando nos aspectos de segurança, transporte, serviços ao redor e potencial de valorização da região.
        Gerar múltiplas consultas de busca usando a ferramenta (google_search), combinando o endereço/bairro/cidade com termos relevantes.
        Utilizar termos que ajudem a encontrar informações de localização e proximidade, muitas vezes extraídas do Google Maps pelo próprio Google Search.
        Consultas essenciais:
        "[endereco_completo] [cidade] pontos de interesse próximos"
        "[endereco_completo] [cidade] escolas próximas"
        "[endereco_completo] [cidade] transporte público"
        "[bairro] [cidade] segurança"
        "[bairro] [cidade] comércio e serviços"
        "[bairro] [cidade] notícias locais"
        "[bairro] [cidade] qualidade de vida"
        "[bairro] [cidade] futuro desenvolvimento"
        A saída deverá ter o seguinte formato:
        Coluna 1: Cidade
        Coluna 2: Bairro
        Coluna 3: Serviços Próximos
        Coluna 4: Avaliação da Segurança
        Coluna 5: Transporte Público
        Coluna 6: Pontos de Interesse Próximos
        Coluna 7: Análise do potencial de valorização
      """
  )

  dados_endereco = f"Propriedades: {propriedades}"
  enderecos = call_agent(localizador, dados_endereco)
  return enderecos


In [55]:
################################################
# --- Agente 3: Analisador de Mercado --- #
################################################
def agente_mercado(enderecos):

  mercado = Agent(
      name = "agente_mercado",
      model=MODEL_ID,
      description="Coletar e analisar dados e tendências do mercado imobiliário específicos da área de um imóvel, fornecendo um contexto financeiro para a avaliação.",
      tools=[google_search],
      instruction="""
        Você atuará como um especialista em análise de mercados imobiliários.
        Utilizar as informações dos endereços para para encontrar informações sobre preços médios, valorização histórica, tendências atuais e notícias relevantes sobre o mercado imobiliário na região utilizando a ferramenta (google_search).
        Sintetizar esses dados e retorná-los de forma estruturada.
        Não é necessário analizar as características do imóvel nem da vizinhança imediata.
        A saída deverá ter o seguinte formato:
        Coluna 1: Cidade (do objeto propriedades)
        Coluna 2: Bairro (do objeto propriedades)
        Coluna 3: Preço Médio
        Coluna 4: Valorização Histórica
        Coluna 5: Tendências
        Coluna 6: Liquidez
        Coluna 7: Notícias relevantes
      """
  )

  dados_mercado = f"Dados de endereços: {enderecos}"
  mercado = call_agent(mercado, dados_mercado)
  return mercado


In [56]:
################################################
# --- Agente 4: Avaliador dos dados        --- #
################################################
def agente_avaliador(propriedades, enderecos, mercado):

  avaliador = Agent(
      name = "agente_avaliador",
      model=MODEL_ID,
      description="Avaliar o quão bem um imóvel específico, com base nas informações coletadas, atende aos critérios e preferências do usuário.",
      instruction="""
         Você atuará como um especialista em avaliação de imóveis.
         Você avaliará todas as informações das propriedades, endereços e do mercado e os criterios iniciais do usuário.
         Você vai processar e comparar esses dados e criar um ranking de melhores opções de compra.
         Você deve ser imparcial e basear a avaliação nos fatos e análises fornecidos.
         Não inventar características do imóvel, dados de localização ou mercado. A avaliação deve refletir o que foi encontrado pelos outros agentes.
         Você Não deve dar conselhos financeiros, legais ou recomendações de compra/venda diretas.
         A saída deverá ter o seguinte formato:
         Coluna 1: ID Imóvel
         Coluna 2: Título do Imóvel
         Coluna 3: Descrição
         Coluna 4: Endereço
         Coluna 5: Cidade
         Coluna 6: Bairro
         Coluna 7: Preço
         Coluna 8: Pontuação (valor decimal com nota mínima 0 e nota máxima 10)
         Coluna 9: Resumo da avaliação
         Coluna 10: Telefone de Contato Vendedor
         Coluna 11: Detalhes da análise da localização
         Coluna 12: Detalhes da análise do mercado
         Coluna 13: Pontos Fortes
         Coluna 14: Pontos Fracos
         Coluna 15: Retorno final do Avaliador
      """
  )

  dados_avaliador = f"Dados das Propriedades: {propriedades}/nDados dos Endereços: {enderecos}/nDados do Mercado: {mercado}"
  avaliacao = call_agent(avaliador, dados_avaliador)
  return avaliacao

In [57]:
################################################
# --- Agente 5: Agente de Recomendação     --- #
################################################
def agente_recomendacao(avaliacao):

  recomendacao = Agent(
      name = "agente_recomendacao",
      model=MODEL_ID,
      description="Processar as avaliações de múltiplos imóveis e gerar uma lista ranqueada das melhores opções para apresentar ao usuário, sintetizando as informações de forma compreensível.",
      instruction="""
        Você irá classificar esses imóveis com base na sua compatibilidade com os critérios do usuário e outros fatores relevantes da avaliação.
        Retornar uma lista ranqueada e sumarizada, colocando em primeiro lugar os imóveis com melhor avaliação e no final imóveis com pior avaliação.
        Basear o ranking e os resumos estritamente no conteúdo das avaliações fornecidas pelo Agente Avaliador.
        Manter um tom informativo e objetivo na apresentação dos resumos, mesmo que a avaliação original contenha um "resumo de compatibilidade".
        A saída deverá ter o seguinte formato:
        Coluna 1: ID Imóvel
        Coluna 2: Título do Imóvel
        Coluna 3: Descrição
        Coluna 4: Endereço
        Coluna 5: Cidade
        Coluna 6: Bairro
        Coluna 7: Preço
        Coluna 8: Pontuação
        Coluna 9: Resumo da recomendação
        Coluna 10: Motivos da recomendação
        Coluna 11: Telefone de Contato Vendedor
      """
  )

  dados_recomendacao = f"Dados das avaliações: {avaliacao}"
  recomendacao = call_agent(recomendacao, dados_recomendacao)
  return recomendacao



In [58]:
print("Seja Bem-Vindo a AImovel+, seu melhor parceiro para ajudar a encontrar imóveis para investimento.\n")
print("A continuação solicitaremos algumas informações para realizarmos a busca e encontrar os melhores imóveis de investimento para você.\n\n")

tipo = input("Qual é o tipo de imóvel que você está procurando? (Ex: casa/apartamento/galpão)")
if not tipo:
  print("Por favor preencher o tipo de imóvel")
else:
  orcamento = input("Qual é o valor do orçamento para a compra do imóvel? (Em Reais)")
  if not orcamento:
    print("Por favor preencher o orçamento")
  else:
    bairro = input("Qual é o bairro que você está avaliando?")
    if not bairro:
      print("Por favor preencher o bairro")
    else:
      cidade = input("Em qual cidade fica esse bairro?")
      if not cidade:
        print("Por favor preencher a cidade")
      else:
        params_adic = input("Você gostaria de incluir alguma outra informação sobre o imóvel para considerar na análise? (Ex: m2, n. de quartos, n. de banheiros, etc.)")
        print("\n")
        print("Início da análise")

        propriedades = agente_busca_propriedade(tipo, orcamento, bairro, cidade, params_adic)
        display(to_markdown(propriedades))
        print("-------------------------------------------------------------------------------------")

        enderecos = agente_localizacao(propriedades)
        display(to_markdown(enderecos))
        print("-------------------------------------------------------------------------------------")

        mercado = agente_mercado(enderecos)
        display(to_markdown(mercado))
        print("-------------------------------------------------------------------------------------")

        avaliacao = agente_avaliador(propriedades, enderecos, mercado)
        display(to_markdown(avaliacao))
        print("-------------------------------------------------------------------------------------")

        recomendacao = agente_recomendacao(avaliacao)
        display(to_markdown(recomendacao))
        print("\nEspero que esta análise ajude na sua decisão de investimento. Muito obrigado e até a próxima!!!")
        print("-------------------------------------------------------------------------------------")





Seja Bem-Vindo a AImovel+, seu melhor parceiro para ajudar a encontrar imóveis para investimento.

A continuação solicitaremos algumas informações para realizarmos a busca e encontrar os melhores imóveis de investimento para você.


Qual é o tipo de imóvel que você está procurando? (Ex: casa/apartamento/galpão)apartamento
Qual é o valor do orçamento para a compra do imóvel? (Em Reais)350000
Qual é o bairro que você está avaliando?Vila Mariana
Em qual cidade fica esse bairro?São Paulo
Você gostaria de incluir alguma outra informação sobre o imóvel para considerar na análise? (Ex: m2, n. de quartos, n. de banheiros, etc.)studios até 25m2 e perto do metrô


Início da análise


> OK. Aqui estão as buscas que farei para encontrar o apartamento ideal:
> 
> 
> Com certeza! Abaixo apresento algumas opções de apartamentos do tipo studio que correspondem aos critérios que você forneceu:
> 
> **Importante:** Os preços e a disponibilidade dos imóveis podem variar. Recomendo entrar em contato diretamente com as imobiliárias ou os corretores responsáveis para obter informações atualizadas e agendar visitas.
> 
> | ID | Título do Imóvel                                                                 | Descrição                                                                                                                               | Preço (R$) | Endereço                                                | Cidade      | Bairro        | Construtora             | Telefone de Contato do Vendedor |
> | -- | --------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- | ---------- | ------------------------------------------------------- | ----------- | ------------- | ----------------------- | ------------------------------- |
> | 1  | Lançamentos de Studio na Vila Mariana (próximo ao metro)                         | Studios de 25m² com varanda, um apartamento compacto, mas com ganho de espaços muito mais confortáveis e de alta qualidade, projetados com design inteligente. | 350.000    | Rua Prof: Sud Mennucci                    | São Paulo   | Vila Mariana  |  Não especificado                         |  Não especificado                               |
> | 2  | Studio na Vila Mariana com 25m² Próximo a Estação do Metro Ana Rosa!              | Studio                                                                    | 450.000    | Próximo a Estação do Metro Ana Rosa                | São Paulo   | Vila Mariana  |  Não especificado                         | Não especificado                                |
> | 3  | Apartamento/Studio A Venda 23m2                                            | Com Lazer Completo,  Lavanderia coletiva, Coworking, Sala de reunião, Central delivery, Lounge externo, Sky bar, FitnessYoga, Solário.                                                                  | 340.950    | Não informado                                                | São Paulo   | Vila Mariana  |  Não especificado                         |  Não especificado                               |
> Espero que esta lista seja um bom ponto de partida para sua busca!


-------------------------------------------------------------------------------------


> Para auxiliar na sua escolha, farei uma análise contextual dos studios listados na Vila Mariana, São Paulo.
> 
> 
> Com base nas informações encontradas, aqui está uma análise contextual dos studios na Vila Mariana, São Paulo, conforme solicitado:
> 
> | Coluna 1: Cidade | Coluna 2: Bairro | Coluna 3: Serviços Próximos | Coluna 4: Avaliação da Segurança | Coluna 5: Transporte Público | Coluna 6: Pontos de Interesse Próximos | Coluna 7: Análise do potencial de valorização |
> |---|---|---|---|---|---|---|
> | São Paulo | Vila Mariana | Escolas renomadas (Colégio Bandeirantes, Liceu Pasteur, ESPM, FMU), hospitais (Hospital Santa Cruz, Sepaco, Unidade Einstein), comércio diversificado (restaurantes, bares, lojas), serviços de saúde, serviços de escritório e apoio administrativo, serviços de tecnologia da informação, serviços de alimentação, arquitetura e engenharia, serviços jurídicos e de contabilidade. | Considerada um bairro de segurança média a alta, com policiamento regular e ruas bem iluminadas. Câmeras de vigilância contribuem para a segurança. | Excelente acesso ao transporte público: estações de metrô Vila Mariana, Ana Rosa e Chácara Klabin, diversas linhas de ônibus, ciclovias e ciclofaixas. | Parque Ibirapuera, SESC Vila Mariana, Museu Lasar Segall, Cinemateca Brasileira, Casa Modernista, Centro de Memórias do Corpo de Bombeiros, Templo Budista Tzong Kwan, Shopping Metrô Santa Cruz. Próximo à Avenida Paulista. | Alta valorização imobiliária, impulsionada pela localização estratégica, infraestrutura completa, mobilidade urbana, oferta de empreendimentos de alto padrão e qualidade de vida. |
> 
> **Análise Detalhada:**
> 
> *   **Serviços Próximos:** A Vila Mariana oferece uma vasta gama de serviços, desde educação de alto nível até saúde, cultura, lazer e comércio diversificado. Isso proporciona conveniência e qualidade de vida aos moradores.
> 
> *   **Avaliação da Segurança:** A segurança na Vila Mariana é geralmente avaliada como boa, com muitos moradores relatando que se sentem seguros no bairro. A presença de policiamento, boa iluminação e monitoramento por câmeras contribuem para essa percepção.
> 
> *   **Transporte Público:** A Vila Mariana possui excelente acesso ao transporte público, com diversas estações de metrô e linhas de ônibus que facilitam a locomoção para outras regiões da cidade.
> 
> *   **Pontos de Interesse Próximos:** A proximidade com o Parque Ibirapuera e outras atrações culturais e de lazer torna a Vila Mariana um local desejável para se viver.
> 
> *   **Análise do Potencial de Valorização:** A Vila Mariana tem apresentado um forte potencial de valorização imobiliária nos últimos anos. Fatores como a localização estratégica, a infraestrutura completa e a qualidade de vida têm impulsionado a demanda por imóveis na região.
> 
> Espero que esta análise detalhada ajude você a tomar uma decisão informada sobre a escolha do seu studio na Vila Mariana.
> 


-------------------------------------------------------------------------------------


...

-------------------------------------------------------------------------------------


> OK. Com base nos dados fornecidos e nas buscas realizadas, aqui está a avaliação dos imóveis:
> 
> | Coluna 1: ID Imóvel | Coluna 2: Título do Imóvel                                                                 | Coluna 3: Descrição                                                                                                                               | Coluna 4: Endereço                                                | Coluna 5: Cidade      | Coluna 6: Bairro        | Coluna 7: Preço      | Coluna 8: Pontuação | Coluna 9: Resumo da avaliação                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Coluna 10: Telefone de Contato Vendedor | Coluna 11: Detalhes da análise da localização                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Coluna 12: Detalhes da análise do mercado                                                                                                                                                                                                                                                                                     | Coluna 13: Pontos Fortes                                                                                                                                                  | Coluna 14: Pontos Fracos                                                                              | Coluna 15: Retorno final do Avaliador                                                                                                                                                                                                                                                                                                   |
> | ------------------ | --------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------- | ----------- | ------------- | --------------- | --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
> | 1                  | Lançamentos de Studio na Vila Mariana (próximo ao metro)                         | Studios de 25m² com varanda, um apartamento compacto, mas com ganho de espaços muito mais confortáveis e de alta qualidade, projetados com design inteligente. | Rua Prof: Sud Mennucci                    | São Paulo   | Vila Mariana  | 350.000         | 8.5             | Este studio apresenta um bom custo-benefício na Vila Mariana, com design inteligente e varanda. A proximidade com o metrô é um grande diferencial. No entanto, faltam informações detalhadas sobre o condomínio e lazer.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Não especificado                      | Localização excelente na Vila Mariana, próximo a serviços, comércio, transporte público e pontos de interesse como o Parque Ibirapuera. Segurança média a alta no bairro, com policiamento regular e ruas bem iluminadas.                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Mercado imobiliário aquecido na Vila Mariana, com alta valorização histórica e tendência de lançamentos de alto padrão e imóveis compactos. Alta liquidez, facilitando a venda ou aluguel do imóvel.                                                                                                                                | Boa localização, design inteligente, varanda, proximidade com o metrô, bom custo-benefício.                                                                                  | Falta de informações sobre o condomínio e lazer, detalhes sobre a construtora, contato do vendedor. | Este studio é uma opção interessante para quem busca um imóvel compacto e bem localizado na Vila Mariana, com bom potencial de valorização. Recomenda-se obter mais informações sobre o condomínio e as opções de lazer antes de tomar uma decisão.                                                                                                                                                                                                                                                                                                    |
> | 2                  | Studio na Vila Mariana com 25m² Próximo a Estação do Metro Ana Rosa!              | Studio                                                                                                                                  | Próximo a Estação do Metro Ana Rosa                | São Paulo   | Vila Mariana  | 450.000         | 7.8             | Este studio tem um preço mais elevado, mas a proximidade com a estação Ana Rosa é um ponto forte. A descrição é vaga, faltando detalhes sobre o imóvel e o condomínio. É necessário buscar mais informações para avaliar o custo-benefício.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Não especificado                      | A localização próxima à estação Ana Rosa oferece excelente acesso ao transporte público e facilidade de locomoção para outras regiões da cidade. A Vila Mariana oferece uma vasta gama de serviços, segurança e qualidade de vida aos moradores.                                                                                                                                                                                                                                                                                                                                                                                                 | O mercado imobiliário na Vila Mariana apresenta alta valorização e liquidez, com lançamentos de empreendimentos de alto padrão e imóveis compactos.                                                                                                                                                                                          | Localização privilegiada próximo ao metrô Ana Rosa.                                                                                                                            | Descrição vaga, falta de detalhes sobre o imóvel e o condomínio, preço mais elevado.              | A localização é excelente, mas a falta de informações dificulta a avaliação do custo-benefício. Recomenda-se obter mais detalhes sobre o imóvel e o condomínio antes de tomar uma decisão.                                                                                                                                                                                                                                                                                                                   |
> | 3                  | Apartamento/Studio A Venda 23m2                                            | Com Lazer Completo, Lavanderia coletiva, Coworking, Sala de reunião, Central delivery, Lounge externo, Sky bar, FitnessYoga, Solário.                                                                                     | Não informado                                                | São Paulo   | Vila Mariana  | 340.950         | 8.0             | Este studio oferece lazer completo e diversas comodidades, como lavanderia coletiva e coworking. A falta de informação sobre o endereço exato dificulta a avaliação da localização. O preço é competitivo, mas é importante verificar a qualidade dos acabamentos e a taxa de condomínio.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Não especificado                      | A falta de informação sobre o endereço exato impede uma análise precisa da localização. É importante verificar a proximidade com serviços, comércio, transporte público e pontos de interesse.                                                                                                                                                                                                                                                                                                                                                                                              | O mercado imobiliário na Vila Mariana apresenta alta valorização e liquidez, com lançamentos de empreendimentos de alto padrão e imóveis compactos.                                                                                                                                                                                          | Lazer completo, lavanderia coletiva, coworking, preço competitivo.                                                                                                        | Falta de informação sobre o endereço exato, qualidade dos acabamentos e taxa de condomínio. | Este studio oferece diversas comodidades e um preço competitivo, mas a falta de informação sobre o endereço exato dificulta a avaliação da localização. Recomenda-se obter mais detalhes sobre o endereço, a qualidade dos acabamentos e a taxa de condomínio antes de tomar uma decisão.                                                                                                                                                                                                                                                                                 |
> 


-------------------------------------------------------------------------------------


> OK. Aqui está a lista ranqueada dos imóveis com base nas avaliações fornecidas:
> 
> | ID Imóvel | Título do Imóvel                                               | Descrição                                                                                                                               | Endereço                                               | Cidade    | Bairro        | Preço     | Pontuação | Resumo da recomendação                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Motivos da recomendação                                                                                                                                      | Telefone de Contato Vendedor |
> | --------- | -------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------ | --------- | ------------- | --------- | --------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------ | ---------------------------- |
> | 1         | Lançamentos de Studio na Vila Mariana (próximo ao metro)       | Studios de 25m² com varanda, um apartamento compacto, mas com ganho de espaços muito mais confortáveis e de alta qualidade, projetados com design inteligente. | Rua Prof: Sud Mennucci                   | São Paulo | Vila Mariana  | 350.000   | 8.5       | Este studio apresenta um bom custo-benefício na Vila Mariana, com design inteligente e varanda. A proximidade com o metrô é um grande diferencial. No entanto, faltam informações detalhadas sobre o condomínio e lazer.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Boa localização, design inteligente, varanda, proximidade com o metrô e bom custo-benefício.                                                                 | Não especificado           |
> | 3         | Apartamento/Studio A Venda 23m2                                 | Com Lazer Completo, Lavanderia coletiva, Coworking, Sala de reunião, Central delivery, Lounge externo, Sky bar, FitnessYoga, Solário.                                                                                     | Não informado                                           | São Paulo | Vila Mariana  | 340.950   | 8.0       | Este studio oferece lazer completo e diversas comodidades, como lavanderia coletiva e coworking. A falta de informação sobre o endereço exato dificulta a avaliação da localização. O preço é competitivo, mas é importante verificar a qualidade dos acabamentos e a taxa de condomínio.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Lazer completo, lavanderia coletiva, coworking e preço competitivo.                                                                                       | Não especificado           |
> | 2         | Studio na Vila Mariana com 25m² Próximo a Estação do Metro Ana Rosa! | Studio                                                                                                                                  | Próximo a Estação do Metro Ana Rosa               | São Paulo | Vila Mariana  | 450.000   | 7.8       | Este studio tem um preço mais elevado, mas a proximidade com a estação Ana Rosa é um ponto forte. A descrição é vaga, faltando detalhes sobre o imóvel e o condomínio. É necessário buscar mais informações para avaliar o custo-benefício.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Localização privilegiada próximo ao metrô Ana Rosa.                                                                                                      | Não especificado           |
> 



Espero que esta análise ajude na sua decisão de investimento. Muito obrigado e até a próxima!!!
-------------------------------------------------------------------------------------
